# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

# Implémenter une régression linéaire 


In [130]:
#importer vos librairies 
import pandas as pds
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt


In [131]:
#charger les données 
#price_availability.csv
#listings_final.csv
#vérifier si tous les individus ont bien un prix

df_price = pds.read_csv("Data/price_availability.csv", sep=";")
df_listings = pds.read_csv("Data/listings_final.csv", sep=";", index_col=0)

In [132]:
df_price.head()

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2


In [133]:
print("Number of rows without price : {}\n".format(df_price.local_price.isna().sum()))

print("Number of rows with null values : {}".format(df_price.isna().sum()))

Number of rows without price : 0

Number of rows with null values : listing_id        0
day               0
created           0
available         0
local_currency    0
local_price       0
min_nights        0
dtype: int64


In [134]:
df_listings

,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00
1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69
2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00
3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48
4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,28335197,Studio cosy Jardin du Luxembourg,entire_home,Paris,Saint Germain des Prés - Odéon,48.848695,2.325857,2,1,0,1.0,False,True,True,False,False,0.79,1.00
996,28583013,Charmant 30m2 - Faubourg Saint Martin,entire_home,Paris,République,48.871623,2.358006,3,1,1,1.0,False,True,True,False,False,1.00,1.00
997,28628316,Cosy flat in the marais - Best area,entire_home,Paris,2e arrondissement,48.867434,2.351771,4,2,1,1.0,False,True,True,True,False,1.00,1.00
998,28792796,Appartement 3 chambres madeleine.,entire_home,Paris,Madeleine - Vendôme,48.870109,2.321475,6,4,2,1.5,False,True,True,False,False,1.00,1.00


In [135]:
print("Total lines : {} ".format(df_listings.listing_id.count()))
print("Number of unique line : {} ".format(df_listings.listing_id.nunique()))

Total lines : 1000 
Number of unique line : 1000 


In [136]:
df_mean = df_price.groupby("listing_id")["local_price"].agg([np.mean])

df_mean

,mean
listing_id,
5396,102.363985
7397,110.107632
9342,396.863014
10010,136.154856
10270,105.779221
...,...
28836096,321.428571
28838519,54.483333
28840013,112.122530


In [137]:
df_mean.rename(columns={"mean" : "mean_price"}, inplace=True)

In [138]:
df_merge = pds.merge(df_listings, df_mean, how="inner", on="listing_id")

df_merge.shape

df_merge

,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor,mean_price
0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00,45.948454
1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69,44.473684
2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00,49.000000
3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48,165.000000
4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90,69.774436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,28335197,Studio cosy Jardin du Luxembourg,entire_home,Paris,Saint Germain des Prés - Odéon,48.848695,2.325857,2,1,0,1.0,False,True,True,False,False,0.79,1.00,38.507732
995,28583013,Charmant 30m2 - Faubourg Saint Martin,entire_home,Paris,République,48.871623,2.358006,3,1,1,1.0,False,True,True,False,False,1.00,1.00,147.752551
996,28628316,Cosy flat in the marais - Best area,entire_home,Paris,2e arrondissement,48.867434,2.351771,4,2,1,1.0,False,True,True,True,False,1.00,1.00,152.860825
997,28792796,Appartement 3 chambres madeleine.,entire_home,Paris,Madeleine - Vendôme,48.870109,2.321475,6,4,2,1.5,False,True,True,False,False,1.00,1.00,49.184211


## Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [139]:
#définir 2 variables de travail
#X := les features à utiliser 
#Y := la target (prix) 
x = df_merge[["person_capacity", "beds", "bedrooms", "bathrooms"]] 
y = df_merge["mean_price"]

In [140]:
#construire l'ensemble de donnée prix 
#
#    INDICE 
# 
# récupérer les prix des ID dans le dataset de prix 
# 🚧 il y a plusieurs prix dans le dataset 🚧

y

0       45.948454
1       44.473684
2       49.000000
3      165.000000
4       69.774436
          ...    
994     38.507732
995    147.752551
996    152.860825
997     49.184211
998     39.503937
Name: mean_price, Length: 999, dtype: float64

En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20 ou 30\%$.

In [141]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#zafficher la shape de vos données 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


In [142]:
(X_train.shape,y_train.shape, X_test.shape, y_test.shape)

((699, 4), (699,), (300, 4), (300,))

## Entraînement

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [154]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(X_train, y_train)

print(X_train)

print(X_test)

print(reg.predict(X_test))

     person_capacity  beds  bedrooms  bathrooms
728                6     3         2        2.0
630                2     1         1        1.0
394                7     4         2        2.0
777                2     1         1        1.0
598                4     2         1        1.0
..               ...   ...       ...        ...
106                6     3         3        1.5
270                2     1         0        1.0
860                2     1         1        1.0
435                4     2         0        1.0
102                5     4         3        2.0

[699 rows x 4 columns]
     person_capacity  beds  bedrooms  bathrooms
453                2     1         1        1.0
793                2     2         1        1.0
209                2     1         0        1.0
309                1     1         1        1.0
740                5     3         2        1.0
..               ...   ...       ...        ...
314                8     3         3        2.5
404             

On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [157]:
#afficher les coefficients
#que remarquez vous ?
 
print("Coef :", reg.coef_)
print("Minimum :", reg.intercept_)
print("Coef accurancy : {} %".format(reg.score(X_test, y_test)*100))


Coef : [ 38.28168742 -21.48146344  20.21008209  88.9106011 ]
Minimum : -40.680503208889576
Coef accurancy : 46.1515323314056 %


## Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [145]:
#faire une prediction sur X


In [146]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2


## Bonus : Analyse de l'homoscédasticité

L'analyse de l'homoscédasticité est primordiale : c'est en particulier elle qui nous permet de vérifier, à partir des résidus, si les bruits vérifient bien l'hypothèse $(\mathcal{H})$. On calcule donc les **résidus studentisés**.

$$t_i^*=\frac{\hat{\varepsilon}_i}{\hat{\sigma}_{(i)} \sqrt{1-h_{ii}}}$$
Avec $h_{ii}=\{X(X^\top X)^{-1} X^\top\}_{ii}=H_{ii}$ la matrice de projection sur l'hyperplan des variables. Plus précisément, $H$ est la matrice qui projette $Y$ sur l'espace engendré par les variables, soit $\hat{Y}=HY$. De même, on considère $\hat{\sigma}_{(i)}$ l'estimateur de la variance du bruit en supprimant l'observation $i$ (par une méthode de validation croisée Leave-One-Out que nous ne détaillerons pas ici).

Dans ce cas, on peut montrer que les résidus studentisés suivent une loi de Student à $n-p-1$ degrés de liberté.

In [156]:
#analyser le code ci-dessous 
import scipy
Y_pred = reg.predict(X_train)
n = X_train.shape[0]
p = 4
residuals = np.abs(y_train - Y_pred)
H = np.matmul(X_train, np.linalg.solve(np.dot(X_train.T, X_train), X_train.T))
std_hat = np.dot(residuals, residuals) / (n - p)
standart_residuals = np.asarray([residuals[i] / np.sqrt(std_hat * (1 - H[i, i])) for i in range(len(residuals))])
student_residuals = np.asarray([ standart_residuals[i] * np.sqrt((n - p - 1) / (n - p - standart_residuals[i]**2)) for i in range(n) ])
cook = np.asarray([ H[i, i] * student_residuals[i] / (X_train.shape[1] * (1 - H[i, i])) for i in range(n) ])

plt.figure(figsize=(20, 12))
plt.subplot(221)
plt.scatter(Y_pred, student_residuals, s=12, c="white", edgecolors="blue")
plt.plot([min(Y_pred), max(Y_pred)], [ scipy.stats.t.ppf(q=0.975, df=n-p-1), scipy.stats.t.ppf(q=0.975, df=n-p-1)], color="green", alpha=0.6, label="Quantile de Student")
plt.title("Analyse de l’homoscédasticité")
plt.xlabel("Prédictions $\hat{y}_i$")
plt.ylabel("Résidus studentisés $|t_i^*|$")
plt.legend()

ValueError: Shape of passed values is (699, 699), indices imply (699, 4)